# L-moment characterization of GOES data

[L-moments](https://en.wikipedia.org/wiki/L-moment) are linear combinations of order statistics that can be used to characterize probability distributions on the real line.  Typicall, the first four L-moments are used, which are analagous to the conventional moments: mean, variance, skewness, and kurtosis. L-moments are more robust than conventional moments.  They can be used to characterize a distribution for descriptive purposes, and they can be used to robustly fit parametric models to data.  However, the latter is complicated by the fact that it is not always straightforward to determine the population values of L-moments in terms of distribution parameters.

In [ ]:
import lmom
import pandas as pd
import numpy as np
from read import get_goes
import matplotlib.pyplot as plt

In [ ]:
df = get_goes(2017)
df["Year"] = df["Time"].dt.year
df["dayofyear"] = df["Time"].dt.dayofyear
df = df.query("dayofyear<=150")

Calculate the raw L-moments for each day of data.

In [ ]:
lm = []
for (k, dv) in df.groupby(["Year", "dayofyear"]):
    v = np.sort(dv["Flux1"].values)
    row = [lmom.l1(v), lmom.l2(v), lmom.l3(v), lmom.l4(v)]
    lm.append(row)

In [ ]:
lm = np.asarray(lm)
lm = pd.DataFrame(lm, columns=["l1", "l2", "l3", "l4"])
lm.head()

Standardized L-moments.

In [ ]:
lm["l3s"] = lm["l3"] / lm["l2"]
lm["l4s"] = lm["l4"] / lm["l2"]

Investigate relationships among the L-moments.

In [ ]:
v = ["l1", "l2", "l3s", "l4s"]
na = ["L-mean", "L-dispersion", "Standardized L-skew", "Standardized L-kurtosis"]
for j in range(4):
    for k in range(j):
        for dolog in [False, True]:

            # Don't log the skew
            if dolog and 2 in [j, k]:
                continue

            plt.clf()
            plt.grid(True)
            if dolog:
                plt.plot(np.log(lm[v[j]]), np.log(lm[v[k]]), "o", alpha=0.5, mfc="none")
                plt.xlabel("log %s" % na[j], size=15)
                plt.ylabel("log %s" % na[k], size=15)
            else:
                plt.plot(lm[v[j]], lm[v[k]], "o", alpha=0.5, mfc="none")
                plt.xlabel(na[j], size=15)
                plt.ylabel(na[k], size=15)
            plt.show()